# EN ESTA NOTEBOOK SE IMPLEMENTARÁN LOS ALGORITMOS DE INTELIGENCIA ARTIFICIAL SUPERVISADOS Y SE EVALUARÁN LOS RESULTADOS PARA EL PROYECTO




## ARBOLES DE DECISIÓN

In [17]:
#importanción de librerías a utilizar

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

#ARBOL DE DECISIÓN
from sklearn.tree import DecisionTreeClassifier

In [18]:
# IMPORTACIÓN DEL DATASET REDUCIDO LUEGO DEL PROCESAMIENTO DE COLUMNAS

df = pd.read_csv('friesgo_2018_reduc.csv',  sep=',', encoding='utf-8')

df.head(5)

,id,region,tamanio_aglomerado,tipo_vivienda,obtiene_gas,obtiene_agua,baño,cant_componentes,tipo_hogar,ingresos,...,consumo_5porc_fv,medido_colesterol,colesterol_alto,consumido_alcohol,consumo_regular_riesgo,dijeron_diabetes,fue_embarazo,familiares_directos_diabetes,familiares_no_directos_diabetes,control_diabetes
0,1128639,1,1,3,1,1,1,2,2,50000,...,2,1,2,1,2.0,2,NaN,2,2,1
1,1709939,1,1,1,1,1,1,3,4,40000,...,1,1,1,1,2.0,2,NaN,2,2,1
2,6874130,1,1,3,1,1,1,2,2,40000,...,2,1,2,1,2.0,2,NaN,2,2,1
3,10319375,1,1,3,4,1,1,1,1,80000,...,2,1,1,1,2.0,2,NaN,2,1,1
4,11140857,1,1,3,1,1,1,2,8,30000,...,2,1,2,1,2.0,2,NaN,2,99,1


In [19]:
df.columns

Index(['id', 'region', 'tamanio_aglomerado', 'tipo_vivienda', 'obtiene_gas',
       'obtiene_agua', 'baño', 'cant_componentes', 'tipo_hogar', 'ingresos',
       'asignaciones', 'sexo', 'edad', 'situacion_conyugal',
       'nivel_instruccion', 'cobertura_salud_nueva', 'ansiedad_depresion',
       'af_intensa_veces_por_semana', 'af_intensa_min_totales_por_semana',
       'af_intensa_min_totales_por_semana_nsnc',
       'af_moderada_veces_por_semana', 'af_moderada_min_totales_por_semana',
       'actividad_fisica_moderada_min_totales_por_semana_nsnc',
       'caminata_veces_por_semana', 'caminata_min_totales_por_semana',
       'caminata_min_totales_por_semana_nsnc', 'tiempo_sentado_min_por_dia',
       'tiempo_sentado_min_por_dia_nsnc',
       'razon_ninguna_actividad_la_ultima_semana',
       'razon_poca_actividad_la_ultima_semana', 'af_fue_actividad_domestica',
       'af_fue_actividad_laboral', 'af_fue_traslado', 'af_fue_deporte',
       'nivel_actividad_fisica', 'barreras_actividad_f

In [20]:
#muevo columnas de variables dependiendtes al final

cols_to_move = [ 'veces_presion_alta', 'colesterol_alto', 'dijeron_diabetes']

df = df[[col for col in df if col not in cols_to_move] + cols_to_move]

print (df)

               id  region  tamanio_aglomerado  tipo_vivienda  obtiene_gas  \
0         1128639       1                   1              3            1   
1         1709939       1                   1              1            1   
2         6874130       1                   1              3            1   
3        10319375       1                   1              3            4   
4        11140857       1                   1              3            1   
...           ...     ...                 ...            ...          ...   
29219  2092256226       6                   4              1            1   
29220  2097771079       6                   4              1            1   
29221  2101464264       6                   4              1            1   
29222  2116612011       6                   4              1            1   
29223  2134624977       6                   4              1            1   

       obtiene_agua  baño  cant_componentes  tipo_hogar  ingresos  ...  \
0

In [21]:
# Variables independientes

#se eliminan algunas columnas que pueden no ser variables independientes

#X_columns = []